<a href="https://colab.research.google.com/github/beingUday/Stock-market-trends-prediction/blob/main/Predictive_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
! pip install xgboost

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,mean_squared_log_error,root_mean_squared_error
from sklearn.ensemble import VotingRegressor,StackingRegressor

In [21]:
actual=pd.read_csv("/content/drive/MyDrive/stocks/Actual_data.csv")
actual

,Date,Open,High,Low,Close,Volume,Tomorrow,Target,Stock name,Yfinance ticker,Zerodha token
0,2010-01-04,29.544373,30.627756,29.544373,30.467913,466064,30.263664,0,Baja Finance,BAJFINANCE.NS,81153
1,2010-01-05,31.085081,31.142802,29.837416,30.263664,318779,30.698782,1,Baja Finance,BAJFINANCE.NS,81153
2,2010-01-06,30.281414,31.169432,29.766363,30.698782,192881,30.512300,0,Baja Finance,BAJFINANCE.NS,81153
3,2010-01-07,31.071754,31.071754,30.103813,30.512300,138876,31.067320,1,Baja Finance,BAJFINANCE.NS,81153
4,2010-01-08,30.556710,31.879856,30.410185,31.067320,312696,31.853207,1,Baja Finance,BAJFINANCE.NS,81153
...,...,...,...,...,...,...,...,...,...,...,...
36985,2024-12-23,817.299988,824.000000,812.200012,821.150024,4933200,812.049988,0,State Bank of India (SBI),SBIN.NS,779521
36986,2024-12-24,824.250000,826.200012,809.200012,812.049988,10602771,812.450012,1,State Bank of India (SBI),SBIN.NS,779521
36987,2024-12-26,813.000000,822.849976,808.000000,812.450012,8318714,799.650024,0,State Bank of India (SBI),SBIN.NS,779521
36988,2024-12-27,816.299988,818.299988,797.200012,799.650024,10649800,788.299988,0,State Bank of India (SBI),SBIN.NS,779521


In [22]:
processed=pd.read_csv("/content/drive/MyDrive/stocks/Processed data.csv")
processed

,Date,Open,High,Low,Close,Volume,Tomorrow,Target,Stock name,Yfinance ticker,Zerodha token
0,2010-01-04,-0.823037,-0.822634,-0.822972,-0.822650,-0.697873,-0.822992,0,Baja Finance,BAJFINANCE.NS,-0.952330
1,2010-01-04,-0.366181,-0.367472,-0.360584,-0.363668,-0.649173,-0.363172,1,Larsen & Toubro (L&T),LT.NS,1.898380
2,2010-01-04,-0.686265,-0.685837,-0.685961,-0.684542,-0.645445,-0.683167,1,Kotak Mahindra Bank,KOTAKBANK.NS,-0.542569
3,2010-01-04,-0.609498,-0.610896,-0.608675,-0.610621,-0.575081,-0.610821,1,Tata Consultancy Services (TCS),TCS.NS,1.911911
4,2010-01-04,-0.659808,-0.660541,-0.658831,-0.659807,-0.402440,-0.659423,1,Infosys,INFY.NS,-0.626309
...,...,...,...,...,...,...,...,...,...,...,...
36975,2024-12-27,1.022123,1.004572,1.032624,1.022248,-0.690432,1.023132,1,Hindustan Unilever (HUL),HINDUNILVR.NS,-0.677369
36976,2024-12-27,-0.195539,-0.200938,-0.203294,-0.208545,0.137110,-0.218026,0,State Bank of India (SBI),SBIN.NS,-0.255864
36977,2024-12-27,0.676294,0.671889,0.690073,0.683332,-0.413244,0.673976,0,Infosys,INFY.NS,-0.626309
36978,2024-12-27,4.614294,4.659776,4.666916,4.668080,-0.690547,4.650157,0,Baja Finance,BAJFINANCE.NS,-0.952330


In [23]:
ds1=pd.read_csv("/content/drive/MyDrive/stocks/Scaled data.csv")
ds1

,Date,Open,High,Low,Close,Volume,Tomorrow,Target,Stock name,Yfinance ticker,Zerodha token
0,2010-01-04,-0.823037,-0.822634,-0.822972,-0.822650,-0.697873,-0.822992,0,Baja Finance,BAJFINANCE.NS,-0.952330
1,2010-01-04,-0.366181,-0.367472,-0.360584,-0.363668,-0.649173,-0.363172,1,Larsen & Toubro (L&T),LT.NS,1.898380
2,2010-01-04,-0.686265,-0.685837,-0.685961,-0.684542,-0.645445,-0.683167,1,Kotak Mahindra Bank,KOTAKBANK.NS,-0.542569
3,2010-01-04,-0.609498,-0.610896,-0.608675,-0.610621,-0.575081,-0.610821,1,Tata Consultancy Services (TCS),TCS.NS,1.911911
4,2010-01-04,-0.659808,-0.660541,-0.658831,-0.659807,-0.402440,-0.659423,1,Infosys,INFY.NS,-0.626309
...,...,...,...,...,...,...,...,...,...,...,...
36975,2024-12-27,1.022123,1.004572,1.032624,1.022248,-0.690432,1.023132,1,Hindustan Unilever (HUL),HINDUNILVR.NS,-0.677369
36976,2024-12-27,-0.195539,-0.200938,-0.203294,-0.208545,0.137110,-0.218026,0,State Bank of India (SBI),SBIN.NS,-0.255864
36977,2024-12-27,0.676294,0.671889,0.690073,0.683332,-0.413244,0.673976,0,Infosys,INFY.NS,-0.626309
36978,2024-12-27,4.614294,4.659776,4.666916,4.668080,-0.690547,4.650157,0,Baja Finance,BAJFINANCE.NS,-0.952330


In [24]:
Y = ds1[["Tomorrow","Target"]]
X = ds1.drop(columns={"Tomorrow","Target","Date","Stock name","Yfinance ticker","Zerodha token"})
Y

,Tomorrow,Target
0,-0.822992,0
1,-0.363172,1
2,-0.683167,1
3,-0.610821,1
4,-0.659423,1
...,...,...
36975,1.023132,1
36976,-0.218026,0
36977,0.673976,0
36978,4.650157,0


In [25]:
X

,Open,High,Low,Close,Volume
0,-0.823037,-0.822634,-0.822972,-0.822650,-0.697873
1,-0.366181,-0.367472,-0.360584,-0.363668,-0.649173
2,-0.686265,-0.685837,-0.685961,-0.684542,-0.645445
3,-0.609498,-0.610896,-0.608675,-0.610621,-0.575081
4,-0.659808,-0.660541,-0.658831,-0.659807,-0.402440
...,...,...,...,...,...
36975,1.022123,1.004572,1.032624,1.022248,-0.690432
36976,-0.195539,-0.200938,-0.203294,-0.208545,0.137110
36977,0.676294,0.671889,0.690073,0.683332,-0.413244
36978,4.614294,4.659776,4.666916,4.668080,-0.690547


In [26]:
X_train = X.iloc[:25889]
X_test = X.iloc[25890:]

In [27]:
X_train

,Open,High,Low,Close,Volume
0,-0.823037,-0.822634,-0.822972,-0.822650,-0.697873
1,-0.366181,-0.367472,-0.360584,-0.363668,-0.649173
2,-0.686265,-0.685837,-0.685961,-0.684542,-0.645445
3,-0.609498,-0.610896,-0.608675,-0.610621,-0.575081
4,-0.659808,-0.660541,-0.658831,-0.659807,-0.402440
...,...,...,...,...,...
25884,0.242789,0.234714,0.240871,0.242127,-0.443024
25885,0.773419,0.756362,0.774839,0.758773,-0.570900
25886,-0.566734,-0.558407,-0.564267,-0.556752,3.038068
25887,0.002819,-0.004480,0.000214,0.000049,0.262158


In [28]:
X_test

,Open,High,Low,Close,Volume
25890,-0.188798,-0.191956,-0.193481,-0.197908,1.412437
25891,1.788363,1.804975,1.683321,1.677427,0.832105
25892,0.757111,0.769996,0.754754,0.782552,-0.461419
25893,0.849911,0.835314,0.823033,0.813029,-0.507054
25894,-0.277203,-0.283332,-0.279853,-0.284406,-0.041071
...,...,...,...,...,...
36975,1.022123,1.004572,1.032624,1.022248,-0.690432
36976,-0.195539,-0.200938,-0.203294,-0.208545,0.137110
36977,0.676294,0.671889,0.690073,0.683332,-0.413244
36978,4.614294,4.659776,4.666916,4.668080,-0.690547


In [29]:
Y_train = Y.iloc[:25889]
Y_test = Y.iloc[25890:]

In [30]:
Y_train

,Tomorrow,Target
0,-0.822992,0
1,-0.363172,1
2,-0.683167,1
3,-0.610821,1
4,-0.659423,1
...,...,...
25884,0.229192,0
25885,0.781736,1
25886,-0.562514,0
25887,0.003528,1


In [31]:
Y_test

,Tomorrow,Target
25890,-0.188863,1
25891,1.772179,1
25892,0.774136,0
25893,0.801320,0
25894,-0.279717,1
...,...,...
36975,1.023132,1
36976,-0.218026,0
36977,0.673976,0
36978,4.650157,0


In [32]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((25889, 5), (11090, 5), (25889, 2), (11090, 2))

In [33]:
def make_pipeline(name, model):
    return Pipeline([
        (name, model)
    ])
def model_Pipeline():
  model_pipelines ={
      "Support Vector Regressor":{
          "pipeline":make_pipeline("SVR",SVR()),
          "param_grid":{
              "SVR__kernel": ['rbf'],
              "SVR__gamma": ["scale", "auto"],
              "SVR__C": [0.5, 1, 5],
              "SVR__epsilon": [0.01, 0.05, 0.1]
          }
      },
      "Random Forest Regressor":{
          "pipeline":make_pipeline("RFR",RandomForestRegressor()),
          "param_grid":{
              'RFR__n_estimators': [i for i in range(0,501,100)],
              'RFR__max_depth': [5, 10, 15, None],
              'RFR__min_samples_split': [2, 5, 10],
              'RFR__min_samples_leaf': [i for i in range(1,10)]
          }
      },
      "Gradient Boosting Regressor":{
          "pipeline":make_pipeline("GBR",GradientBoostingRegressor()),
          "param_grid":{
              'GBR__n_estimators': [i for i in range(0,501,100)],
              'GBR__learning_rate': [0.01, 0.05, 0.1,0.5],
              'GBR__max_depth': [3, 4, 5, 10],
              'GBR__subsample': [0.8, 1.0],
              "GBR__criterion":["friedman_mse","squared_error"]
          }
      },
      "XG Boosting Regressor":{
          "pipeline":make_pipeline("XGBR",XGBRegressor()),
          "param_grid":{
              'XGBR__n_estimators': [100, 200, 300],
              'XGBR__max_depth': [3, 4, 5, 6, 7],
              'XGBR__learning_rate': [0.01, 0.05, 0.1, 0.2],
              'XGBR__subsample': [0.6, 0.8, 1.0],
              'XGBR__colsample_bytree': [0.6, 0.8, 1.0],
              'XGBR__gamma': [0, 0.1, 0.2, 0.3],
              'XGBR__reg_alpha': [0, 0.01, 0.1],
              'XGBR__reg_lambda': [1, 1.5, 2],
          }
      }
      }


  return model_pipelines

In [34]:
def evaluate(X_train,X_test,Y_train,Y_test):
  results=[]
  models=model_Pipeline()
  for name,config in models.items():
    print(f"{name} is being tuned...\n")
    rscv=RandomizedSearchCV(
        estimator=config['pipeline'],
        param_distributions=config['param_grid'],
        cv=5,
        scoring="neg_mean_squared_error",
        random_state=42)
    rscv.fit(X_train,Y_train["Tomorrow"])
    best_model=rscv.best_estimator_
    y_pred=best_model.predict(X_test)

    MAE=mean_absolute_error(y_pred,Y_test["Tomorrow"])
    print(f"Mean_absolute_error:{MAE}")
    MSE=mean_squared_error(y_pred,Y_test["Tomorrow"])
    print(f"Mean_squared_error:{MSE}")
    MSLE=mean_squared_log_error(y_pred,Y_test["Tomorrow"])
    print(f"Mean_squared_log_error:{MSLE}")
    R2=r2_score(y_pred,Y_test["Tomorrow"])
    print(f"R2_score:{R2}")
    RMSE=root_mean_squared_error(y_pred,Y_test["Tomorrow"])
    print(f"Root_mean_squared_error:{RMSE}")
    print("-------------------------------------------------------------------------")
    results.append({
        "Model":name,
        "Best_estimator":best_model,
        "Predictions":list(y_pred),
        "Absolute Error":MAE,
        "Squared Error":MSE,
        "Squared Log Error":MSLE,
        "R2 scores":R2,
        "Root mean squared error":RMSE
    })

  return pd.DataFrame(results).set_index("Model")

In [ ]:
results_ds=evaluate(X_train,X_test,Y_train,Y_test)
results_ds

Support Vector Regressor is being tuned...

Mean_absolute_error:0.3215491772844136
Mean_squared_error:1.2649143339681297
Mean_squared_log_error:0.13005999665169815
R2_score:-1.124273077461294
Root_mean_squared_error:1.1246841040790652
-------------------------------------------------------------------------
Random Forest Regressor is being tuned...

Mean_absolute_error:0.1545790986610796
Mean_squared_error:0.2462057822109731
Mean_squared_log_error:0.010889822873281257
R2_score:0.7449408744609984
Root_mean_squared_error:0.49619127583117895
-------------------------------------------------------------------------
Gradient Boosting Regressor is being tuned...



In [ ]:
results_ds.to_csv("results_ds.csv")
best_models={results_ds["Model"]:results_ds["Best_estimator"]for idx, row in results_ds.reset_index().iterrows()}

In [ ]:
voting_model = VotingRegressor([
    ("RFR", best_models["Random Forest Regressor"]),
    ("GBR", best_models["Gradient Boosting Regressor"]),
    ("XGBR", best_models["XG Boosting Regressor"])

])

voting_model.fit(X_train, Y_train["Tomorrow"])
voting_preds = voting_model.predict(X_test)


In [ ]:
stacking_model = StackingRegressor(
    estimators=[
        ("RFR", best_models["Random Forest Regressor"]),
        ("GBR", best_models["Gradient Boosting Regressor"]),
        ("XGBR", best_models["XG Boosting Regressor"])
    ],
    final_estimator=RandomForestRegressor(n_estimators=100, random_state=42),
    passthrough=True
)

stacking_model.fit(X_train, Y_train["Tomorrow"])
stacking_preds = stacking_model.predict(X_test)

In [ ]:
def evaluate_ensemble(name, model, y_true, y_pred):
    return {
        "Model": name,
        "Best_estimator": model,
        "Predictions": list(y_pred),
        "Absolute Error": mean_absolute_error(y_pred, y_true),
        "Squared Error": mean_squared_error(y_pred, y_true),
        "Squared Log Error": mean_squared_log_error(y_pred, y_true),
        "R2 scores": r2_score(y_pred, y_true),
        "Root mean squared error": root_mean_squared_error(y_pred, y_true)
    }

voting_results = evaluate_ensemble("Voting Regressor", voting_model, Y_test["Tomorrow"], voting_preds)
stacking_results = evaluate_ensemble("Stacking Regressor", stacking_model, Y_test["Tomorrow"], stacking_preds)


results_ds = pd.concat([results_ds, pd.DataFrame([voting_results, stacking_results]).set_index("Model")])


In [ ]:
v_data=pd.read_csv("/content/drive/MyDrive/stocks/Extracted.csv",index_col=0)
v_data